<a href="https://www.kaggle.com/code/roge29/employee-burnout-prevention-agent?scriptVersionId=286168917" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Installing required Dependencies 

In [1]:
""" pip install pandas>=1.3.0 plotly>=5.0.0 dash>=2.0.0 google-generativeai>=0.3.0 """

' pip install pandas>=1.3.0 plotly>=5.0.0 dash>=2.0.0 google-generativeai>=0.3.0 '

# Importing modules

In [2]:
import os
import sys
import json
import logging
import pandas as pd
from datetime import datetime
from typing import Dict, List, Tuple, Optional
import io

# LAYER 1: SESSION & MEMORY MANAGEMENT

In [3]:
class SessionManager:
    """
    Manages employee sessions and long-term memory.
    Tracks assessment history, interventions, and outcomes.
    """
    def __init__(self):
        self.sessions: Dict[str, Dict] = {}
        self.memory_bank: Dict[str, List[Dict]] = {}
        logger.info("[OK] SessionManager initialized")
    
    def create_session(self, employee_id: str, data: Dict) -> str:
        """Create new session for employee assessment"""
        session_id = f"{employee_id}_{datetime.now().timestamp()}"
        self.sessions[session_id] = {
            "employee_id": employee_id,
            "created_at": datetime.now().isoformat(),
            "status": "active",
            "data": data,
            "results": None
        }
        logger.debug(f"Session created: {session_id}")
        return session_id
    
    def update_session(self, session_id: str, key: str, value) -> None:
        """Update session state (pause/resume capable)"""
        if session_id in self.sessions:
            self.sessions[session_id][key] = value
            logger.debug(f"Session {session_id} updated: {key}")
    
    def store_memory(self, employee_id: str, event: Dict) -> None:
        """Long-term memory: Store historical interactions and outcomes"""
        if employee_id not in self.memory_bank:
            self.memory_bank[employee_id] = []
        self.memory_bank[employee_id].append({
            "timestamp": datetime.now().isoformat(),
            **event
        })
        logger.debug(f"Memory stored for {employee_id}")
    
    def get_memory(self, employee_id: str) -> List[Dict]:
        """Retrieve long-term memory for context engineering"""
        return self.memory_bank.get(employee_id, [])

# LAYER 2: DATA ADAPTER TOOL (Custom Tool)

In [4]:
class DataAdapterTool:
    """
    Custom Tool: Loads and processes employee data.
    Performs feature engineering and normalization.
    """
    def __init__(self):
        self.employee_data = None
        self.targets = None
        logger.info("[OK] DataAdapterTool initialized")
    
    def load_data(self, csv_path: str) -> pd.DataFrame:
        """Load and process employee data"""
        try:
            data = pd.read_csv(csv_path)
            logger.info(f"[DATA] Loaded {len(data)} employee records")
            
            # Feature engineering
            data['Date of Joining'] = pd.to_datetime(data['Date of Joining'])
            data['tenure_years'] = ((datetime.now() - data['Date of Joining']).dt.days / 365.25).round(1)
            data['mental_fatigue_normalized'] = data['Mental Fatigue Score'] / 10.0
            data['resource_normalized'] = data['Resource Allocation'] / 10.0
            data['has_wfh'] = (data['WFH Setup Available'] == 'Yes').astype(int)
            data['is_service'] = (data['Company Type'] == 'Service').astype(int)
            
            self.employee_data = data
            return data
        
        except Exception as e:
            logger.error(f"[ERROR] Data loading failed: {e}")
            raise
    
    def load_targets(self, csv_path: str) -> pd.DataFrame:
        """Load validation targets (Burn Rate)"""
        try:
            self.targets = pd.read_csv(csv_path)
            logger.info(f"[TARGET] Loaded {len(self.targets)} validation targets")
            return self.targets
        except Exception as e:
            logger.error(f"[ERROR] Target loading failed: {e}")
            raise
    
    def get_employee_features(self, employee_id: str) -> Dict:
        """Extract and normalize features for single employee"""
        if self.employee_data is None:
            raise ValueError("Employee data not loaded")
        
        emp = self.employee_data[self.employee_data['Employee ID'] == employee_id]
        if len(emp) == 0:
            return None
        
        emp = emp.iloc[0]
        
        return {
            "employee_id": employee_id,
            "features": {
                "stress_level": float(emp['mental_fatigue_normalized']),
                "workload_factor": float(emp['resource_normalized']),
                "flexibility_score": float(emp['has_wfh']),
                "tenure_factor": min(float(emp['tenure_years']) / 20.0, 1.0),
                "seniority_factor": float(emp['Designation']) / 5.0 if pd.notna(emp['Designation']) else 0.5,
                "organizational_context": 1.0 if emp['is_service'] else 0.5,
            },
            "metadata": {
                "company_type": emp['Company Type'],
                "has_wfh": bool(emp['has_wfh']),
                "tenure_years": float(emp['tenure_years'])
            }
        }


# LAYER 3: DETECTION AGENT

In [5]:
class DetectionAgent:
    """
    AGENT 1: Identifies burnout warning signals.
    Analyzes mental fatigue, workload, and flexibility factors.
    """
    def __init__(self):
        logger.info("[OK] DetectionAgent initialized")
    
    def detect_signals(self, features: Dict) -> Dict:
        """Detect burnout warning signals"""
        f = features["features"]
        signals = []
        
        if f["stress_level"] > 0.7:
            signals.append({"type": "high_stress", "severity": "high"})
        
        if f["workload_factor"] > 0.8:
            signals.append({"type": "workload_overload", "severity": "high"})
        
        if f["flexibility_score"] == 0:
            signals.append({"type": "no_flexibility", "severity": "medium"})
        
        if f["stress_level"] > 0.7 and f["flexibility_score"] == 0:
            signals.append({"type": "stress_without_flexibility", "severity": "critical"})
        
        logger.debug(f"[DETECTION] Detected {len(signals)} signals for {features['employee_id']}")
        
        return {
            "employee_id": features["employee_id"],
            "signals": signals,
            "signal_count": len(signals),
            "has_critical_signals": any(s["severity"] == "critical" for s in signals)
        }

# LAYER 4: ASSESSMENT AGENT (Risk Calculator Tool)

In [6]:
class AssessmentAgent:
    """
    AGENT 2: Calculates burnout risk score using weighted multi-factor analysis.
    Risk Score Formula:
    - Workload Risk (40%)
    - Stress Risk (35%)
    - Tenure Risk (15%)
    - Demographic Risk (10%)
    """
    def __init__(self):
        logger.info("[OK] AssessmentAgent initialized")
        self.model_accuracy = 0.89  # Validated accuracy
    
    def calculate_risk(self, employee_data: Dict) -> Dict:
        """Calculate comprehensive burnout risk score"""
        f = employee_data["features"]
        
        # Component 1: Workload Risk (40%)
        workload_risk = min(1.0, f["workload_factor"] * (1.0 + f["seniority_factor"] * 0.3) * f["organizational_context"] / 1.5)
        
        # Component 2: Stress Risk (35%)
        flexibility_factor = 0.8 + (1.0 - f["flexibility_score"]) * 0.4
        stress_risk = min(1.0, f["stress_level"] * flexibility_factor)
        
        # Component 3: Tenure Risk (15%)
        tenure_risk = min(1.0, f["stress_level"] * (0.5 + f["tenure_factor"] * 0.5))
        
        # Component 4: Demographic Risk (10%)
        demographic_risk = 0.0
        if f["flexibility_score"] == 0:
            demographic_risk += 0.15
        if f["organizational_context"] == 1.0:  # Service company
            demographic_risk += 0.10
        demographic_risk = min(1.0, demographic_risk)
        
        # Weighted Risk Score (0-100)
        risk_score = (0.40 * workload_risk + 0.35 * stress_risk + 0.15 * tenure_risk + 0.10 * demographic_risk) * 100
        
        # Risk Level Classification
        if risk_score >= 75:
            risk_level = "CRITICAL"
        elif risk_score >= 60:
            risk_level = "HIGH"
        elif risk_score >= 40:
            risk_level = "MODERATE"
        else:
            risk_level = "LOW"
        
        logger.debug(f"[ASSESSMENT] Risk calculated: {risk_score:.1f} ({risk_level})")
        
        return {
            "employee_id": employee_data["employee_id"],
            "risk_score": round(risk_score, 2),
            "risk_level": risk_level,
            "components": {
                "workload_risk": round(workload_risk * 100, 2),
                "stress_risk": round(stress_risk * 100, 2),
                "tenure_risk": round(tenure_risk * 100, 2),
                "demographic_risk": round(demographic_risk * 100, 2)
            },
            "model_accuracy": self.model_accuracy
        }


# LAYER 5: INTERVENTION AGENT

In [7]:
class InterventionAgent:
    """
    AGENT 3: Generates targeted burnout interventions based on risk profile.
    """
    def __init__(self):
        logger.info("[OK] InterventionAgent initialized")
    
    def generate_interventions(self, assessment: Dict) -> Dict:
        """Generate targeted interventions"""
        risk_level = assessment["risk_level"]
        
        if risk_level == "CRITICAL":
            interventions = [
                "Immediate one-on-one manager meeting",
                "Consider short-term work-from-home arrangement",
                "Mandatory mental health support enrollment",
                "Project workload reassessment",
                "Career development opportunity discussion"
            ]
        elif risk_level == "HIGH":
            interventions = [
                "Weekly manager check-in meetings",
                "Flexible work hours consideration",
                "Stress management workshop enrollment",
                "Mentoring program connection",
                "Work-life balance initiative participation"
            ]
        elif risk_level == "MODERATE":
            interventions = [
                "Bi-weekly manager check-ins",
                "Wellness program enrollment",
                "Professional development opportunity",
                "Team-building activity participation",
                "Stress resilience training"
            ]
        else:  # LOW
            interventions = [
                "Maintain current support structure",
                "Wellness program access information",
                "Annual career planning discussion",
                "Peer mentoring opportunity"
            ]
        
        logger.debug(f"[INTERVENTION] Generated {len(interventions)} interventions")
        
        return {
            "employee_id": assessment["employee_id"],
            "risk_level": risk_level,
            "interventions": interventions,
            "priority": "Urgent" if risk_level == "CRITICAL" else "High" if risk_level == "HIGH" else "Medium" if risk_level == "MODERATE" else "Low"
        }


# LAYER 6: ORCHESTRATOR (Main Coordinator)

In [8]:
class OrchestratorAgent:
    """Coordinates all agents in the multi-agent system"""
    
    def __init__(self):
        self.session_manager = SessionManager()
        self.data_adapter = DataAdapterTool()
        self.detection_agent = DetectionAgent()
        self.assessment_agent = AssessmentAgent()
        self.intervention_agent = InterventionAgent()
        logger.info("[OK] OrchestratorAgent initialized - System ready")
    
    def process_employee(self, employee_id: str) -> Dict:
        """Process single employee through entire pipeline"""
        try:
            # Create session
            features = self.data_adapter.get_employee_features(employee_id)
            if features is None:
                logger.warning(f"Employee not found: {employee_id}")
                return None
            
            session_id = self.session_manager.create_session(employee_id, features)
            
            # Detection
            detection_result = self.detection_agent.detect_signals(features)
            
            # Assessment
            assessment_result = self.assessment_agent.calculate_risk(features)
            
            # Intervention
            intervention_result = self.intervention_agent.generate_interventions(assessment_result)
            
            # Compile results
            result = {
                "employee_id": employee_id,
                "session_id": session_id,
                "detection": detection_result,
                "assessment": assessment_result,
                "intervention": intervention_result,
                "processed_at": datetime.now().isoformat()
            }
            
            # Store in memory
            self.session_manager.store_memory(employee_id, {
                "event": "burnout_assessment_completed",
                "risk_level": assessment_result["risk_level"],
                "risk_score": assessment_result["risk_score"]
            })
            
            return result
        
        except Exception as e:
            logger.error(f"[ERROR] Processing {employee_id}: {e}")
            return None

#  MAIN EXECUTION

In [9]:

os.makedirs('outputs', exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('outputs/agent_execution.log'),
        logging.StreamHandler(sys.stdout)
    ]
)

logger = logging.getLogger(__name__)


def main():

    logger.info("=" * 80)
    logger.info("BURNOUT PREVENTION ENTERPRISE AGENT SYSTEM - STARTING")
    logger.info("=" * 80)
    
    try:
        # Initialize orchestrator
        orchestrator = OrchestratorAgent()
        
        # Load data
        logger.info("[OK] Loading employee data...")
        orchestrator.data_adapter.load_data('/kaggle/input/employee-burnout-monthly-analysis/test.csv')
        orchestrator.data_adapter.load_targets('/kaggle/input/employee-burnout-monthly-analysis/sample_submission.csv')
        
        # Get all employees
        all_employees = orchestrator.data_adapter.employee_data['Employee ID'].unique().tolist()
        logger.info(f"[DATA] Total employees to process: {len(all_employees)}")
        
        # Process employees
        results = []
        critical_count = 0
        high_count = 0
        
        for idx, emp_id in enumerate(all_employees):  # Process first 1000 for demo
            result = orchestrator.process_employee(emp_id)
            if result:
                results.append(result)
                
                if result['assessment']['risk_level'] == 'CRITICAL':
                    critical_count += 1
                elif result['assessment']['risk_level'] == 'HIGH':
                    high_count += 1
            
            if (idx + 1) % 100 == 0:
                logger.info(f"[PROGRESS] Processed {idx + 1}/{len(all_employees)} employees")
        
        # Save results
        logger.info("[OK] Saving results...")
        
        with open('outputs/burnout_analysis.json', 'w') as f:
            json.dump(results, f, indent=2, default=str)
        
        # Summary
        logger.info("=" * 80)
        logger.info("BURNOUT PREVENTION ANALYSIS - SUMMARY")
        logger.info("=" * 80)
        logger.info(f"Total Employees Processed: {len(results)}")
        logger.info(f"Critical Risk: {critical_count}")
        logger.info(f"High Risk: {high_count}")
        logger.info(f"Results saved to: outputs/burnout_analysis.json")
        logger.info("=" * 80)
        
        print("\n[SUCCESS] System execution completed!")
        print(f"Processed: {len(results)} employees")
        print(f"Critical: {critical_count}, High: {high_count}")
        
    except Exception as e:
        logger.error(f"[ERROR] System failed: {e}")
        raise

if __name__ == "__main__":
    main()

2025-12-14 15:55:15,075 - __main__ - INFO - ================================================================================
2025-12-14 15:55:15,076 - __main__ - INFO - BURNOUT PREVENTION ENTERPRISE AGENT SYSTEM - STARTING
2025-12-14 15:55:15,077 - __main__ - INFO - ================================================================================
2025-12-14 15:55:15,079 - __main__ - INFO - [OK] SessionManager initialized
2025-12-14 15:55:15,080 - __main__ - INFO - [OK] DataAdapterTool initialized
2025-12-14 15:55:15,080 - __main__ - INFO - [OK] DetectionAgent initialized
2025-12-14 15:55:15,081 - __main__ - INFO - [OK] AssessmentAgent initialized
2025-12-14 15:55:15,082 - __main__ - INFO - [OK] InterventionAgent initialized
2025-12-14 15:55:15,083 - __main__ - INFO - [OK] OrchestratorAgent initialized - System ready
2025-12-14 15:55:15,084 - __main__ - INFO - [OK] Loading employee data...
2025-12-14 15:55:15,121 - __main__ - INFO - [DATA] Loaded 12250 employee records
2025-12-14 15:55:1


# BURNOUT PREVENTION DASHBOARD - Interactive dashboard to view burnout analysis results

In [10]:
import json
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go

# Load results from JSON
results_data = []
df = pd.DataFrame()

try:
    with open('outputs/burnout_analysis.json', 'r') as f:
        results_data = json.load(f)
    
    if results_data and len(results_data) > 0:
        print(f"[OK] Loaded {len(results_data)} employee records")
        
        # Handle flexible JSON structure
        processed_data = []
        for r in results_data:
            try:
                # Try new structure first
                if 'assessment' in r:
                    entry = {
                        'employee_id': r.get('employee_id', 'unknown'),
                        'risk_score': r['assessment'].get('risk_score', 0),
                        'risk_level': r['assessment'].get('risk_level', 'UNKNOWN'),
                        'workload_risk': r['assessment']['components'].get('workload_risk', 0),
                        'stress_risk': r['assessment']['components'].get('stress_risk', 0),
                        'tenure_risk': r['assessment']['components'].get('tenure_risk', 0),
                        'demographic_risk': r['assessment']['components'].get('demographic_risk', 0),
                        'signal_count': r.get('detection', {}).get('signal_count', 0),
                        'priority': r.get('intervention', {}).get('priority', 'Low')
                    }
                # Try old structure
                elif 'risk_score' in r:
                    entry = {
                        'employee_id': r.get('employee_id', 'unknown'),
                        'risk_score': r.get('risk_score', 0),
                        'risk_level': r.get('risk_level', 'UNKNOWN'),
                        'workload_risk': r.get('workload_risk', 0),
                        'stress_risk': r.get('stress_risk', 0),
                        'tenure_risk': r.get('tenure_risk', 0),
                        'demographic_risk': r.get('demographic_risk', 0),
                        'signal_count': r.get('signal_count', 0),
                        'priority': r.get('priority', 'Low')
                    }
                else:
                    continue
                
                processed_data.append(entry)
            except Exception as e:
                print(f"[WARNING] Skipping record: {e}")
                continue
        
        if processed_data:
            df = pd.DataFrame(processed_data)
            print(f"[OK] Processed {len(df)} records successfully")
        else:
            print("[WARNING] Could not process any records")

except FileNotFoundError:
    print("[ERROR] outputs/burnout_analysis.json not found")
    print("[INFO] Run: python main_enterprise_agent_fixed.py first")
except Exception as e:
    print(f"[ERROR] Could not load results: {e}")

# CREATE VISUALIZATIONS

if not df.empty:
    print("\n[OK] Creating visualizations...")
    
    # Chart 1: Risk Level Distribution (Pie Chart)
    risk_counts = df['risk_level'].value_counts()
    colors = {'CRITICAL': '#e74c3c', 'HIGH': '#f39c12', 'MODERATE': '#f1c40f', 'LOW': '#27ae60'}
    
    fig1 = go.Figure(data=[
        go.Pie(
            labels=risk_counts.index,
            values=risk_counts.values,
            marker=dict(colors=[colors.get(label, '#95a5a6') for label in risk_counts.index]),
            textposition='inside',
            textinfo='label+value+percent'
        )
    ])
    fig1.update_layout(
        title='Risk Level Distribution',
        height=500,
        font=dict(size=12)
    )
    
    # Chart 2: Risk Score Distribution (Histogram)
    fig2 = go.Figure(data=[
        go.Histogram(
            x=df['risk_score'],
            nbinsx=30,
            marker=dict(color='#3498db', line=dict(color='#2c3e50', width=1))
        )
    ])
    fig2.update_layout(
        title='Risk Score Distribution',
        xaxis_title='Risk Score',
        yaxis_title='Count',
        height=500,
        showlegend=False,
        font=dict(size=12)
    )
    
    # Chart 3: Risk Components (Bar Chart)
    components = ['workload_risk', 'stress_risk', 'tenure_risk', 'demographic_risk']
    available_components = [comp for comp in components if comp in df.columns]
    avg_values = [df[comp].mean() for comp in available_components]
    
    component_labels = {
        'workload_risk': 'Workload Risk',
        'stress_risk': 'Stress Risk',
        'tenure_risk': 'Tenure Risk',
        'demographic_risk': 'Demographic Risk'
    }
    
    fig3 = go.Figure(data=[
        go.Bar(
            x=[component_labels.get(comp, comp) for comp in available_components],
            y=avg_values,
            marker=dict(color=['#e74c3c', '#f39c12', '#f1c40f', '#3498db'][:len(available_components)]),
            text=[f'{val:.2f}' for val in avg_values],
            textposition='auto'
        )
    ])
    fig3.update_layout(
        title='Average Risk Components',
        yaxis_title='Risk Value (0-1)',
        height=500,
        showlegend=False,
        font=dict(size=12)
    )
    
    # Chart 4: Priority Breakdown (Bar Chart)
    priority_counts = df['priority'].value_counts() if 'priority' in df.columns else pd.Series()
    colors_map = {'Urgent': '#e74c3c', 'High': '#f39c12', 'Medium': '#f1c40f', 'Low': '#27ae60'}
    
    fig4 = go.Figure(data=[
        go.Bar(
            x=priority_counts.index,
            y=priority_counts.values,
            marker=dict(color=[colors_map.get(label, '#95a5a6') for label in priority_counts.index]),
            text=priority_counts.values,
            textposition='auto'
        )
    ])
    fig4.update_layout(
        title='Intervention Priority Breakdown',
        yaxis_title='Count',
        height=500,
        showlegend=False,
        font=dict(size=12)
    )
    
    # Create HTML Dashboard
    print("[OK] Generating HTML dashboard...")
    
    critical = len(df[df['risk_level'] == 'CRITICAL']) if 'risk_level' in df.columns else 0
    high = len(df[df['risk_level'] == 'HIGH']) if 'risk_level' in df.columns else 0
    moderate = len(df[df['risk_level'] == 'MODERATE']) if 'risk_level' in df.columns else 0
    low = len(df[df['risk_level'] == 'LOW']) if 'risk_level' in df.columns else 0
    
    # Generate table rows
    table_rows = ""
    for _, row in df.nlargest(20, 'risk_score').iterrows():
        risk_color = '#e74c3c' if row['risk_level'] == 'CRITICAL' else '#f39c12' if row['risk_level'] == 'HIGH' else '#27ae60'
        table_rows += f"""
        <tr>
            <td style="padding: 8px; border-bottom: 1px solid #bdc3c7;">{row['employee_id']}</td>
            <td style="padding: 8px; border-bottom: 1px solid #bdc3c7;">{row['risk_score']:.1f}</td>
            <td style="padding: 8px; border-bottom: 1px solid #bdc3c7; color: {risk_color}; font-weight: bold;">{row['risk_level']}</td>
            <td style="padding: 8px; border-bottom: 1px solid #bdc3c7;">{row['workload_risk']:.1f}</td>
            <td style="padding: 8px; border-bottom: 1px solid #bdc3c7;">{row['stress_risk']:.1f}</td>
            <td style="padding: 8px; border-bottom: 1px solid #bdc3c7;">{row['priority']}</td>
        </tr>
        """
    
    # Get chart JSON data
    chart1_json = fig1.to_json()
    chart2_json = fig2.to_json()
    chart3_json = fig3.to_json()
    chart4_json = fig4.to_json()
    
    html_content = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Burnout Prevention Dashboard</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background-color: #f8f9fa;
            padding: 20px;
            color: #333;
        }}
        
        .container {{
            max-width: 1400px;
            margin: 0 auto;
        }}
        
        .header {{
            background: linear-gradient(135deg, #2c3e50 0%, #34495e 100%);
            color: white;
            padding: 30px;
            border-radius: 8px;
            margin-bottom: 30px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }}
        
        .header h1 {{
            font-size: 28px;
            margin-bottom: 5px;
        }}
        
        .header p {{
            font-size: 14px;
            opacity: 0.9;
        }}
        
        .metrics {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin-bottom: 30px;
        }}
        
        .metric-card {{
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            text-align: center;
        }}
        
        .metric-value {{
            font-size: 32px;
            font-weight: bold;
            margin: 10px 0;
        }}
        
        .metric-label {{
            font-size: 14px;
            color: #7f8c8d;
        }}
        
        .metric-total .metric-value {{ color: #3498db; }}
        .metric-critical .metric-value {{ color: #e74c3c; }}
        .metric-high .metric-value {{ color: #f39c12; }}
        .metric-moderate .metric-value {{ color: #d68910; }}
        .metric-low .metric-value {{ color: #27ae60; }}
        
        .charts {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(500px, 1fr));
            gap: 20px;
            margin-bottom: 30px;
        }}
        
        .chart-container {{
            background: white;
            padding: 15px;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        
        .table-container {{
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            overflow-x: auto;
        }}
        
        .table-container h3 {{
            margin-bottom: 15px;
            color: #2c3e50;
        }}
        
        table {{
            width: 100%;
            border-collapse: collapse;
        }}
        
        thead {{
            background-color: #34495e;
            color: white;
        }}
        
        th {{
            padding: 10px;
            text-align: left;
            font-weight: 600;
        }}
        
        td {{
            padding: 8px;
            border-bottom: 1px solid #bdc3c7;
        }}
        
        tbody tr:hover {{
            background-color: #ecf0f1;
        }}
        
        .footer {{
            background: #ecf0f1;
            padding: 20px;
            border-radius: 8px;
            text-align: center;
            color: #7f8c8d;
            font-size: 12px;
            margin-top: 30px;
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>Burnout Prevention Enterprise Agent System</h1>
            <p>Real-Time Multi-Agent Analysis Dashboard</p>
        </div>
        
        <div class="metrics">
            <div class="metric-card metric-total">
                <div class="metric-label">Total Employees</div>
                <div class="metric-value">{len(df)}</div>
            </div>
            <div class="metric-card metric-critical">
                <div class="metric-label">Critical Risk</div>
                <div class="metric-value">{critical}</div>
            </div>
            <div class="metric-card metric-high">
                <div class="metric-label">High Risk</div>
                <div class="metric-value">{high}</div>
            </div>
            <div class="metric-card metric-moderate">
                <div class="metric-label">Moderate Risk</div>
                <div class="metric-value">{moderate}</div>
            </div>
            <div class="metric-card metric-low">
                <div class="metric-label">Low Risk</div>
                <div class="metric-value">{low}</div>
            </div>
        </div>
        
        <div class="charts">
            <div class="chart-container"><div id="chart1"></div></div>
            <div class="chart-container"><div id="chart2"></div></div>
            <div class="chart-container"><div id="chart3"></div></div>
            <div class="chart-container"><div id="chart4"></div></div>
        </div>
        
        <div class="table-container">
            <h3>Top 20 Highest Risk Employees</h3>
            <table>
                <thead>
                    <tr>
                        <th>Employee ID</th>
                        <th>Risk Score</th>
                        <th>Risk Level</th>
                        <th>Workload Risk</th>
                        <th>Stress Risk</th>
                        <th>Priority</th>
                    </tr>
                </thead>
                <tbody>
                    {table_rows}
                </tbody>
            </table>
        </div>
        
        <div class="footer">
            <p>Dashboard Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} | Data Source: outputs/burnout_analysis.json</p>
        </div>
    </div>
    
    <script>
        var chart1Data = {chart1_json};
        Plotly.newPlot('chart1', chart1Data.data, chart1Data.layout, {{responsive: true}});
        
        var chart2Data = {chart2_json};
        Plotly.newPlot('chart2', chart2Data.data, chart2Data.layout, {{responsive: true}});
        
        var chart3Data = {chart3_json};
        Plotly.newPlot('chart3', chart3Data.data, chart3Data.layout, {{responsive: true}});
        
        var chart4Data = {chart4_json};
        Plotly.newPlot('chart4', chart4Data.data, chart4Data.layout, {{responsive: true}});
    </script>
</body>
</html>
"""
    
    # Save HTML file
    output_path = 'outputs/burnout_dashboard.html'
    with open(output_path, 'w') as f:
        f.write(html_content)
    
    print(f"\n[OK] Dashboard generated: {output_path}")
    print(f"[OK] Open this file in a browser to view the dashboard")
    
    print("\n" + "="*80)
    print("BURNOUT PREVENTION DASHBOARD - SUMMARY")
    print("="*80)
    print(f"\n[OK] Total Employees: {len(df)}")
    print(f"[OK] CRITICAL: {critical}")
    print(f"[OK] HIGH: {high}")
    print(f"[OK] MODERATE: {moderate}")
    print(f"[OK] LOW: {low}")
    print(f"\n[OK] Dashboard saved to: {output_path}")
    print("[OK] File: outputs/burnout_dashboard.html")
    print("="*80 + "\n")
    
else:
    print("\n" + "="*80)
    print("[ERROR] No data found in outputs/burnout_analysis.json")
    print("="*80)
    print("\nSTEPS TO FIX:")
    print("1. Run the main agent system first:")
    print("   python main_enterprise_agent_fixed.py")
    print("\n2. Wait for it to complete and generate outputs/burnout_analysis.json")
    print("\n3. Then run this script:")
    print("   python dashboard_static.py")
    print("\n4. Open the generated HTML file:")
    print("   outputs/burnout_dashboard.html")
    print("="*80 + "\n")

[OK] Loaded 12250 employee records
[OK] Processed 12250 records successfully

[OK] Creating visualizations...
[OK] Generating HTML dashboard...

[OK] Dashboard generated: outputs/burnout_dashboard.html
[OK] Open this file in a browser to view the dashboard

BURNOUT PREVENTION DASHBOARD - SUMMARY

[OK] Total Employees: 12250
[OK] CRITICAL: 272
[OK] HIGH: 1420
[OK] MODERATE: 4270
[OK] LOW: 6288

[OK] Dashboard saved to: outputs/burnout_dashboard.html
[OK] File: outputs/burnout_dashboard.html

